In [2]:
import librosa
import librosa.display
import os
import numpy as np
from collections import Counter
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Mapping from strings to integers for predictions
lab = {'reggae':0,
       'rock':1,
       'country':2,
       'disco':3,
       'hiphop':4,
       'classical':5,
       'metal':6,
       'blues':7,
       'jazz':8,
       'pop':9
}      
nBatch = 6
batchDim = list()
batchIndex = list()
nsong = 100
currentIdx = 0
for i in range(nBatch):
  nBatchSong =int(nsong / nBatch)
  nBatch -= 1
  batchDim.append(nBatchSong)
  nsong -= nBatchSong

  nextIdx = currentIdx + nBatchSong -1
  idxs = (currentIdx, nextIdx)

  currentIdx += nBatchSong
  batchIndex.append(idxs)

#example with nBatch = 6
#batchDim = [17,17,17,17,16,16]
#batchIndex =[(0,16),(17,33),(34,50),(51,67),(68,83),(84,99)]
# 1) 17songs/genre indexes -> (0-16)
# 2) 17songs/genre indexes -> (17-33)
# 3) 17songs/genre indexes -> (34-50)
# 4) 17songs/genre indexes -> (51-67)
# 5) 16songs/genre indexes -> (68-83)
# 6) 16songs/genre indexes -> (84-99)

def loadBatch(index):
  """
  Samples all the songs applying the sliding window approach and returns two np arrays,
  one with all the data extracted and one with the label associated to the segment of the song

  param index: the index of the partition of the dataset
  """
  data = []
  labels = []
  dataset_path = "/content/drive/MyDrive/genres"

  l = list()

  segments_per_track = 21
  overlapping = 0.75
  track_length = 30 #seconds
  sample_length = 5 #seconds

  for i,(dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    if dirpath is not dataset_path:
      label = dirpath.split('/')[-1]
      print("Loading %s"%label)
      for i,f in enumerate(filenames):
        if i>=batchIndex[index][0] and i<=batchIndex[index][1]:
          file_path = os.path.join(dirpath,f)
          signal, sample_rate = librosa.load(file_path)
          truncated = 0
          if len(signal) < 661500: # if the track is less than 30 seconds i dont take the last segment
            l.append(label)
            truncated = 1
          # Samples all the segments of the song and adds them to an array
          for s in range(segments_per_track - truncated ):
            start_sample_index = int(s * (1-overlapping) * sample_length * sample_rate) #s = 0 -> 0, s = 1 -> int(27562.5) = 27562
            end_sample_index = int(start_sample_index + sample_length * sample_rate-1) #s = 0 -> 27561 , s = 1 -> 55123
            sample = signal[start_sample_index:end_sample_index+1]
            data.append(sample)
            labels.append(lab[label])
  return np.array(data),np.array(labels)

In [6]:
filter = [(2,128), (6,256), (1,512)] # Defines the sequence of filters to apply to the ANN
kernelS = 3
poolS = 3
stride = [3, 1] # Defines the possible values of the stride to use in the ANN

In [ ]:
# Loads the selected partition of the dataset for training and validation
x_train, y_train=loadBatch(5)
x_valid, y_valid=loadBatch(0)

In [12]:
import time
def unison_shuffled_copies(a, b):
    """
    Generates a random permutation of the value of a and b and applies it to them
    """
    np.random.seed(int(time.time()))
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [13]:
x_train, y_train = unison_shuffled_copies(x_train, y_train)
x_valid, y_valid = unison_shuffled_copies(x_valid, y_valid)

In [ ]:
import soundfile as sf
#check if everything correct
sample_rate = 22050
sf.write('prova.wav', x_train[100,:],sample_rate)

print(Counter(y_train))
y_train[100]

In [ ]:
# Residual Layer
def res1d(input, nFilters, kernelSize, stride):
  """
  Creates the residual layer and returns it
  
  param input: the output of the previous layer in the network
  param nFilters: the dimension of the filters in the Conv1D
  param kernelSize: the dimension of the kernel in Conv1D
  param stride: the dimension of the stride in Conv1D
  """
  y = keras.layers.Conv1D(filters = nFilters, kernel_size = kernelSize, strides = stride, padding = "same")(input)  
  y = keras.layers.BatchNormalization()(y)
  y = keras.layers.LeakyReLU()(y)
  y = keras.layers.Conv1D(filters = nFilters, kernel_size = kernelSize, strides = stride, padding = "same")(y)
  y = keras.layers.BatchNormalization()(y)
  # if the shape of the shortcut and y aren't equal, we add a convolutional1D layer and a batch normalization to the shortcut
  if input.shape[2] != y.shape[2]:
    shortcut = keras.layers.Conv1D(filters = nFilters, kernel_size = kernelSize, strides = stride, padding = "same")(input)  
    shortcut = keras.layers.BatchNormalization()(shortcut)
  else:
    shortcut = input
  y = keras.layers.Add()([shortcut, y])
  y = keras.layers.LeakyReLU()(y)
  return y


In [ ]:
# Creation of the Input Layer:
x = keras.Input(shape=(110250,1))

# Creation of the first Convolutional Layer:
y = keras.layers.Conv1D(filters=filter[0][1], kernel_size = kernelS, strides = stride[0], padding = "same")(x)

# Creation of all the Residual layers and the maxPooling
for filterType in filter:
  for i in range(filterType[0]):
    y = res1d(y, filterType[1], kernelS, stride[1])
    print(y.shape)
    y = keras.layers.MaxPooling1D(pool_size = poolS, strides = stride[0])(y)
    print(y.shape)

# Creation of the last convolutional layer
y = keras.layers.Conv1D(filters=filter[2][1], kernel_size = 1, strides = stride[1], padding = "same")(y)

# Creation of the of the output layer with activation function "Softmax"
y = keras.layers.Flatten()(y)
predictions = keras.layers.Dense(10, activation='softmax')(y)

In [ ]:
#model = keras.Model(inputs=x, outputs=predictions)
model = keras.models.load_model("/content/drive/MyDrive/Training6batches/5Train0Valid/model.h5")

In [ ]:
# Compiles the model assigning a loss funciton , optimizer and a metric
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam', metrics=keras.metrics.sparse_categorical_accuracy)


In [ ]:
model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 4)
callbacks_list = [early_stop]

In [ ]:
n_epochs = 100
n_batch = 32
# Trains the model  
model.fit(x_train, y_train,validation_data = (x_valid, y_valid), epochs=n_epochs, batch_size=n_batch, callbacks=callbacks_list )

In [ ]:
# Saves the trained model in a file
model.save("/content/drive/MyDrive/Training6batches/5Train0Valid/model.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
x_test, y_test=loadBatch(5)

Loading reggae
Loading rock
Loading country
Loading disco
Loading hiphop
Loading classical
Loading metal
Loading blues
Loading jazz
Loading pop


In [ ]:
y_hat = model.predict(x_test)

In [ ]:
#Testing
accuracy_test = keras.metrics.SparseCategoricalAccuracy()(y_test,y_hat)
print('Accuracy (test dataset):%1.2f%%'% (accuracy_test * 100))

Accuracy (test dataset):87.93%


In [ ]:
#Aggregation of the segments in the songs
song=list()
for genre in range(10):
  seg = y_hat[y_test==genre,:]
  pos = 0
  resto = len(seg)%21
  for i in range(int(len(seg)/21)):
    counts = np.mean(seg[pos*21:pos*21 + 21],axis=0)
    pos += 1
    song.append(counts)
  if resto > 0:
    counts = np.mean(seg[pos*21:],axis=0)
    song.append(counts)


songTrue=list()
for genre in range(10):
  seg = y_test[y_test==genre]
  pos = 0
  resto = len(seg)%21
  for i in range(int(len(seg)/21)):
    counts = np.bincount(seg[pos*21:pos*21 + 21])
    pos += 1
    songTrue.append(np.argmax(counts))
  if resto > 0:
    counts = np.bincount(seg[pos*21:])
    songTrue.append(np.argmax(counts))

In [ ]:
#Testing Aggregate
accuracy_test = keras.metrics.SparseCategoricalAccuracy()(songTrue,song)
print('Accuracy (test dataset):%1.2f%%'% (accuracy_test * 100))

Accuracy (test dataset):90.00%
